history = {'loss': [59.09539031982422, 20.74762535095215, 6.167580604553223, 1.1619025468826294, 0.4605521261692047, 0.21871888637542725, 0.1192503497004509, 0.07793787121772766, 0.054312363266944885, 0.0407898835837841, 0.028131674975156784, 0.021961461752653122, 0.019648274406790733], 'accuracy': [0.20116618275642395, 0.34110787510871887, 0.5218659043312073, 0.7142857313156128, 0.8629737496376038, 0.9533527493476868, 0.9970845580101013, 0.9970845580101013, 0.9970845580101013, 0.9970845580101013, 0.9970845580101013, 1.0, 1.0], 'val_loss': [36.71835708618164, 18.216588973999023, 4.106420516967773, 1.6931058168411255, 1.2400538921356201, 1.0574349164962769, 1.0227702856063843, 0.8964598178863525, 0.9650161266326904, 0.9169053435325623, 1.0857079029083252, 0.9495127201080322, 0.9930201172828674], 'val_accuracy': [0.15662650763988495, 0.3012048304080963, 0.42168673872947693, 0.4457831382751465, 0.5783132314682007, 0.6144578456878662, 0.5662650465965271, 0.6987951993942261, 0.6385542154312134, 0.6385542154312134, 0.6385542154312134, 0.6024096608161926, 0.6265060305595398], 'lr': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.00020000001, 0.00020000001]}

In [ ]:
# Multi CNN + Edge + Frequent Colours
# 69.87951993942261


import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Lambda
from tensorflow.keras.models import Model
_dataset_directory = "drive/MyDrive/DL_PROJECT_DATASET_V1"

# Define CNN structure
def create_cnn(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    return Model(inputs=input_layer, outputs=x)
def create_cnn_for_colors(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (1, 1), activation='relu')(input_layer)  # Using smaller kernel size
    x = Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

# Create inputs for each image type
input_original = Input(shape=(224, 224, 3))
input_edges = Input(shape=(224, 224, 1))
input_colors = Input(shape=(5, 3))  # Assuming 5 prominent colors

# Create CNNs
cnn_original = create_cnn((224, 224, 3))
cnn_edges = create_cnn((224, 224, 1))
cnn_colors = create_cnn_for_colors((5, 3, 1))  # Note: This architecture might need adjustment

# Get outputs from CNNs
output_original = cnn_original(input_original)
output_edges = cnn_edges(input_edges)
output_colors = cnn_colors(Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_colors))

# Concatenate outputs
concatenated = Concatenate()([output_original, output_edges, output_colors])

# Dense layers for classification
dense = Dense(128, activation='relu')(concatenated)
output_layer = Dense(6, activation='softmax')(dense)  # Assuming 6 classes

# Complete model
model = Model(inputs=[input_original, input_edges, input_colors], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

import cv2
import numpy as np
from sklearn.cluster import KMeans

def preprocess_image(img, n_colors=5):
    # Convert from float32 to uint8 and from RGB to BGR
    image = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    
    # Convert to grayscale for edge detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Edge detection
    edges = cv2.Canny(gray_image, 100, 200)
    resized_edges = cv2.resize(edges, (224, 224))

    # Top n colors
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_colors, n_init=10, random_state=0).fit(pixels)
    prominent_colors = kmeans.cluster_centers_.astype(int)

    resized_image = cv2.resize(image, (224, 224))
    return resized_image, resized_edges, prominent_colors

def custom_generator(image_data_generator, steps_per_epoch):
    batch_count = 0
    while True:
        for batch_x, batch_y in image_data_generator:
            batch_x_original = np.zeros((batch_x.shape[0], 224, 224, 3))
            batch_x_edges = np.zeros((batch_x.shape[0], 224, 224, 1))
            batch_x_colors = np.zeros((batch_x.shape[0], 5, 3))

            for i, img in enumerate(batch_x):
                original, edges, colors = preprocess_image(img)
                batch_x_original[i] = cv2.cvtColor(original, cv2.COLOR_BGR2RGB) / 255.0
                batch_x_edges[i] = np.expand_dims(edges, axis=-1) / 255.0
                batch_x_colors[i] = colors / 255.0

            yield [batch_x_original, batch_x_edges, batch_x_colors], batch_y

            batch_count += 1
            if batch_count >= steps_per_epoch:
                batch_count = 0
                break

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

total_train_samples = 343
total_val_samples = 83
batch_size = 32

train_steps = total_train_samples // batch_size + (1 if total_train_samples % batch_size else 0)
val_steps = total_val_samples // batch_size + (1 if total_val_samples % batch_size else 0)

train_data_gen = train_datagen.flow_from_directory(
    _dataset_directory, 
    target_size=(224, 224), 
    batch_size=batch_size, 
    class_mode='categorical', 
    subset='training'
)

val_data_gen = validation_datagen.flow_from_directory(
    _dataset_directory, 
    target_size=(224, 224), 
    batch_size=batch_size, 
    class_mode='categorical', 
    subset='validation'
)

train_generator = custom_generator(train_data_gen, train_steps)
validation_generator = custom_generator(val_data_gen, val_steps)


from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model_V3.0.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# ModelCheckpoint to save the model after every epoch
checkpoint = ModelCheckpoint(
    'art_style_model_best_v3.0.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    mode='min'
)

# EarlyStopping to stop training when the validation loss has not improved after 5 epochs
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=1, 
    mode='min',
    restore_best_weights=True
)

# ReduceLROnPlateau to reduce the learning rate when the validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    verbose=1, 
    mode='min',
    min_lr=0.00001
)


history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=val_steps,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Save the trained model
model.save('art_style_model_v3.0.h5')

print(history.history)

